# Mastermind by Dexter

## Reference Materials
* [Mastermind (board game) - Wikipedia](https://en.wikipedia.org/wiki/Mastermind_(board_game))
* [enum — Support for enumerations — Python 3.9.1 documentation](https://docs.python.org/3/library/enum.html)
* [re — Regular expression operations — Python 3.9.1 documentation](https://docs.python.org/3/library/re.html)
* [9. Classes — Python 3.9.1 documentation](https://docs.python.org/3/tutorial/classes.html)
* [5. Data Structures — Python 3.9.1 documentation](https://docs.python.org/3/tutorial/datastructures.html)

## Game Rules
1. There are six colored pegs. 
1. The pattern to be decoded contains 4 pegs of any color, and repetitions are allowed.
1. Every guess will be responded with key peg of _white_ and _black_ color, which correspond to\
a matching color peg but at wrong location, and both matching color and position.
1. Player wins when guess has the exact same pattern both in terms of color and position.
1. Otherwise, game over after 13 failed guesses.

## Implementation
* This game is human player play against computer generated game.
* Since we are running in text enviornment, key pegs are instead notated as such:
    - {X} for both correct color and position for one guessed peg;
    - {O} for correct color but wrong position peg;
    - {_} for remaining incorrect guesses.
* The six color we have are _red, orange, yellow, blue, green, and purple_.\
For faster game play, the input only accept *R*, *O*, *Y*, *B*, *G*, and *P* correspondingly.

## Game Setup
Firstly, create enum to represent color to make code more readible.

Made a convenience function **fromInt(n)** to aid generating random color.

In [1]:
from enum import Enum
class Color(Enum):
    RED = 1
    ORANGE = 2
    YELLOW = 3
    GREEN = 4
    BLUE = 5
    PURPLE = 6
    
    def fromInt(n):
        if n > 6:
            n = n % 6
        
        return Color(n)
    
    def fromChar(c):
        if c == 'R':
            return Color.RED
        elif c == 'O':
            return Color.ORANGE
        elif c == 'Y':
            return Color.YELLOW
        elif c == 'G':
            return Color.GREEN
        elif c == 'B':
            return Color.BLUE
        elif c == 'P':
            return Color.PURPLE
        else:
            return None
            
    
    def __repr__(self):
        if self == Color.RED:
            return 'RED'
        elif self == Color.ORANGE:
            return 'ORANGE'
        elif self == Color.YELLOW:
            return 'YELLOW'
        elif self == Color.GREEN:
            return 'GREEN'
        elif self == Color.BLUE:
            return 'BLUE'
        else:
            return 'PURPLE'

In [2]:
Color.fromInt(7) # should return Color.Red

RED

In [3]:
from random import randint

In [4]:
global pattern

SLOT = 4 # each pattern has 4 slots
def generatePattern():
    pattern = []
    for _ in range(SLOT):
        randomColor = Color.fromInt(randint(1, 6))
        pattern.append(randomColor)
        
    return pattern

In [5]:
test_pattern = generatePattern()
print(test_pattern)

[PURPLE, ORANGE, GREEN, GREEN]


## Game Logic

Now we have our pattern, how about we make our own _educated_ guess and verify the result?

Before that, lets create another enum to better represent result key peg status!

In [6]:
class Result(Enum):
    EXACT = 1 # both color and position are correct
    COLOR_ONLY = 2
    WRONG = 3
    
    def __repr__(self):
        if self == Result.EXACT:
            return '[X]'
        elif self == Result.COLOR_ONLY:
            return '[O]'
        else:
            return '[ ]'

So with the mean to represent our results, we can now work on the logic/algorithm\
to verify result.

Firstly, here are our _very educated_ guesses:

In [7]:
first_try = [Color.RED, Color.YELLOW, Color.GREEN, Color.BLUE]

We can verify the result in 2 parts.

Firstly the easier part, find how many exact matches are there.

In [8]:
def exactMatch(pattern, guess):
    pairs = zip(pattern, guess)
    exactMatches = [p for p in pairs if p[0] == p[1]]
    return len(exactMatches)

In [9]:
exactMatch(test_pattern, first_try)

1

In [10]:
exactMatch(test_pattern, test_pattern)

4

Second count how many common colors are there in our guess and the hidden pattern.

In [11]:
from copy import deepcopy

def commonColor(pattern, guess):
    guess_copy = deepcopy(guess) # what will happen if we dont copy guess here?
    for p in pattern:
        for g in guess_copy:
            if p == g:
                guess_copy.remove(g)
                break
    
    return SLOT - len(guess_copy)

Finally, we can collect those two intermediate results and report the final verdict.

Remember, exact matches are also instances of common color matches.

In [12]:
def feedback(pattern, guess):
    exact = exactMatch(pattern, guess)
    common = commonColor(pattern, guess) - exact
    wrong = SLOT - exact - common
    
    if exact + common > 4:
        print('Fatal error: more than 4 feedback matches.')
        sys.exit()
    
    feedback = [Result.EXACT] * exact + [Result.COLOR_ONLY] * common + [Result.WRONG] * wrong
    
    return feedback

In [13]:
print(test_pattern)
print(first_try)
feedback(test_pattern, first_try)

[PURPLE, ORANGE, GREEN, GREEN]
[RED, YELLOW, GREEN, BLUE]


[[X], [ ], [ ], [ ]]

Now we are halfway done.

## Game Flow

The pseudo code of our game flow is as following:
```
    WHILE guessCount <= MAX_GUESS:
        get input
        validate input
        verify result
        print game board
        win / continue / lose!?
```
Let's learn how to read player input properly, using regular expression:

In [14]:
import re

def validGuess(input):
    result = re.match("^[ROYGBP]{4}$|^surrender$", input)
    return result != None

```^[ROYGBP]{4}$|^surrender$```

What does this weird line mean?

1. **^** means the beginning of a string, and **$** means the end
2. we can choose any one (and only one at a time) charater from **[ROYGBP]**
3. and have exactly **{4}** of those choices.
4. **|** means OR
5. we can match with a string begin with "surrender", and nothing after.

Let's observe our validator behaviour

In [15]:
validGuess("RRRR")

True

In [16]:
validGuess("RBGYP")

False

In [17]:
validGuess("XXXX")

False

In [18]:
validGuess("surrender")

True

In [19]:
validGuess(" surrender")

False

In [20]:
validGuess("surrender ")

False

In [21]:
validGuess(" RYBG")

False

In [22]:
validGuess("RYBG ")

False

In [23]:
class GameState(Enum):
    VICTORY = 1
    DEFEAT = 2
    CONTINUE = 3
    
class Mastermind:
    MAX_GUESS = 13
    
    def __init__(self, pattern):
        self.pattern = pattern
        self.guesses = []
        self.feedbacks = []
        
    def __repr__(self):
        printable = ""
        for guess, result in zip(self.guesses, self.feedbacks): 
            printable += str(guess) + "|" + str(result) + "\n"
            
        printable += "[ ][ ][ ][ ]|[[ ][ ][ ][ ]]\n" * (self.MAX_GUESS - len(self.guesses))
        
        return printable
    
    def newGuess(self, guess):
        self.guesses.append(guess)
        self.feedbacks.append(feedback(self.pattern, guess))
    
    def status(self):
        if len(self.feedbacks) > 0 and self.feedbacks[-1] == [Result.EXACT] * 4:
            return GameState.VICTORY
        
        if len(self.guesses) >= self.MAX_GUESS:
            return GameState.DEFEAT
        
        return GameState.CONTINUE

In [27]:

def newGame():
    pattern = generatePattern()

    game = Mastermind(pattern)

    # terminate the guessing process if gameState is VICTORY or DEFEAT
    while game.status() == GameState.CONTINUE:
        # get input
        guess_input = input()
        
        # validate input
        while not validGuess(guess_input):
            guess_input = input()
                
        guess = list(map(Color.fromChar, list(guess_input))) # what is this!

        # verify result
        game.newGuess(guess)

        # print game board
        print(game)
    # end of guessing proess

    # Print end game status
    if game.status() == GameState.VICTORY:
        print("You win!")
    else:
        print("You lose.")
        print("The Pattern is ", game.pattern)

In [25]:
print(list(map(Color.fromChar, list("RRRR"))))

[RED, RED, RED, RED]


## Main Program

In [28]:
def validContinue(input):
    result = re.match("^[YN]$", input)
    return result != None

def main():
    while True:
        newGame()
        
        continue_input = input("Continue? [Y/N]")
        while not validContinue(continue_input):
            continue_input = input("Continue? [Y/N]")
        
        if continue_input == "N":
            print("Goodbye!")
            break

if __name__ == '__main__':
    main()

 RRRR


[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]



 RRRR


[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]



 RRRR


...

[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[ ][ ][ ][ ]|[[ ][ ][ ][ ]]



 RRRR


[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]
[RED, RED, RED, RED]|[[X], [X], [ ], [ ]]

You lose.
The Pattern is  [RED, RED, BLUE, GREEN]


Continue? [Y/N] N


Goodbye!


## Further Improvements